<h1 style = 'color: orange'> Configurações </h1>

### Importação da Bibliotecas

In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By

import pandas as pd
import re
import datetime
import time
from sqlalchemy import create_engine

### Conectar ao Banco de dados

In [2]:
conexao = create_engine('postgresql://{user}:{password}@{host}:{port}/{database}'.format(
    host='db.yeqkmhmarjrfnxyocpnh.supabase.co', 
    port = 5432,
    database='postgres',
    user='postgres', 
    password='ThButewicz2409'
))

### Configuração do Selenium

In [3]:
options = Options()
options.add_experimental_option("detach",True)

driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()),
                          options=options)

#### Obter os dados

In [4]:
def get_info(url, acao):
    # Abrir a URL
    driver.get(url)
    time.sleep(1.8)
    
    dados = {}
    cards = driver.find_elements(By.CLASS_NAME, 'd-inline-block')

    dados['acao'] = acao
    dados['valor'] = driver.find_elements(By.CLASS_NAME, 'value')[0].text

    session = driver.find_element(By.CLASS_NAME,'indicator-today-container')

    for item in session.find_elements(By.CLASS_NAME,'item'):
        indicador = item.find_element(By.TAG_NAME,'h3').text
        valor = item.find_element(By.TAG_NAME,'strong').text
        dados[indicador] = valor

    return pd.DataFrame([dados])

In [5]:
arquivo = pd.read_csv('acoes.csv',sep=';')

In [6]:
nacional = True

In [7]:
df_original = pd.DataFrame()

for acao in arquivo['TICKER']:
    if nacional == True:
        url = f'https://statusinvest.com.br/acoes/{acao}'
    else:
        url = f'https://statusinvest.com.br/acoes/eua/{acao}'
    
    df_original = df_original.append(get_info(url, acao))

C:\Users\zThan\AppData\Local\Temp\ipykernel_2592\2694780989.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_original = df_original.append(get_info(url, acao))
C:\Users\zThan\AppData\Local\Temp\ipykernel_2592\2694780989.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_original = df_original.append(get_info(url, acao))
C:\Users\zThan\AppData\Local\Temp\ipykernel_2592\2694780989.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_original = df_original.append(get_info(url, acao))
C:\Users\zThan\AppData\Local\Temp\ipykernel_2592\2694780989.py:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_original = df_original.append(g

In [22]:
df = df_original.copy()

## Manipular o DataFrame

In [23]:
def get_number(string):
    match = re.search(r'[\d,]+', string)
    
    if match:
         return float(match.group(0).replace(',', '.'))
    else:
        return None

In [24]:
for coluna in df.iloc[:,1:].columns:
    # Converter tudo pra FLOAT
    df[coluna] = df[coluna].apply(lambda string: get_number(string))

In [25]:
df.columns

Index(['acao', 'valor', 'D.Y', 'P/L', 'PEG RATIO', 'P/VP', 'EV/EBITDA',
       'EV/EBIT', 'P/EBITDA', 'P/EBIT', 'VPA', 'P/ATIVO', 'LPA', 'P/SR',
       'P/CAP. GIRO', 'P/ATIVO CIRC. LIQ.', 'DÍV. LÍQUIDA/PL',
       'DÍV. LÍQUIDA/EBITDA', 'DÍV. LÍQUIDA/EBIT', 'PL/ATIVOS',
       'PASSIVOS/ATIVOS', 'LIQ. CORRENTE', 'M. BRUTA', 'M. EBITDA', 'M. EBIT',
       'M. LÍQUIDA', 'ROE', 'ROA', 'ROIC', 'GIRO ATIVOS',
       'CAGR RECEITAS 5 ANOS', 'CAGR LUCROS 5 ANOS'],
      dtype='object')

In [26]:
df = df[['acao','valor','D.Y','P/L','P/VP','EV/EBITDA','PEG RATIO','CAGR RECEITAS 5 ANOS','CAGR LUCROS 5 ANOS','M. EBITDA','ROE','ROIC','DÍV. LÍQUIDA/EBITDA','LIQ. CORRENTE']]

## Visualização

In [27]:
def good_value():
    return 'background-color: #2E924C; font-weight: bold'

def medium_value():
    return 'background-color: #FFFFE5; color:black; opacity: 95%'

def bad_value():
    return 'background-color: red; font-weight: bold'

In [28]:
df.columns

Index(['acao', 'valor', 'D.Y', 'P/L', 'P/VP', 'EV/EBITDA', 'PEG RATIO',
       'CAGR RECEITAS 5 ANOS', 'CAGR LUCROS 5 ANOS', 'M. EBITDA', 'ROE',
       'ROIC', 'DÍV. LÍQUIDA/EBITDA', 'LIQ. CORRENTE'],
      dtype='object')

In [29]:
df = df.reset_index(drop=True)

In [30]:
df = df.fillna(0)

In [31]:
df_view = (
df.style
            # Ajustar as casas decimais
            .format('{:.2f}', na_rep='MISS',subset=[coluna  for coluna in df.columns if coluna not in ('acao')])
            .format('{:.2f}%', na_rep='MISS',subset=['M. EBITDA','ROE','ROIC','CAGR RECEITAS 5 ANOS','CAGR LUCROS 5 ANOS'])
            # Indicadores de Preço
            .applymap(lambda v: good_value() if (v > 0) and (v < 20)  else medium_value(), subset=['P/L'])
            .applymap(lambda v: good_value() if (v > 2)  else (medium_value() if v >= 0 and v <= 2 else bad_value()), subset=['P/VP'])
             #Indicadores de Crescimento
            .applymap(lambda v: good_value() if (v < 1) and (v > 0)  else (medium_value() if v >= 2 and v <= 2 else bad_value()),subset=['PEG RATIO'])
            # Rentabilidade
            .background_gradient(cmap='YlGn',subset=['M. EBITDA','ROE','ROIC'])
            # Indicadores de Individamentos
            .applymap(lambda v: good_value() if (v < 1)  else (medium_value() if v >= 1 and v<= 3 else bad_value()), 
                      subset=['DÍV. LÍQUIDA/EBITDA'])
)

In [33]:
df_view

,acao,valor,D.Y,P/L,P/VP,EV/EBITDA,PEG RATIO,CAGR RECEITAS 5 ANOS,CAGR LUCROS 5 ANOS,M. EBITDA,ROE,ROIC,DÍV. LÍQUIDA/EBITDA,LIQ. CORRENTE
0,BAUH4,70.00,1.73,95.65,3.81,26.27,0.66,3.33%,0.00%,6.30%,3.98%,3.45%,1.35,1.20
1,BEEF11,3.62,0.00,6.26,5.36,4.70,0.12,20.68%,0.00%,9.18%,85.67%,14.51%,2.90,1.50
2,BEEF3,7.93,6.96,13.68,11.72,4.70,0.25,20.68%,0.00%,9.18%,85.67%,14.51%,2.90,1.50
3,BRFS3,10.85,0.00,5.31,1.99,22.69,0.04,13.70%,0.00%,2.77%,37.45%,1.11%,10.59,1.10
4,BSEV3,9.00,0.00,10.22,10.84,2.04,0.06,10.88%,0.00%,47.91%,106.05%,30.90%,0.86,0.39
5,CAML3,7.61,3.78,5.48,0.89,0.00,0.02,16.96%,14.10%,0.00%,16.16%,10.72%,0.00,1.99
6,JALL3,8.45,5.21,3.23,1.30,0.00,0.00,0.00%,0.00%,0.00%,40.16%,16.71%,0.00,2.53
7,JBSS3,18.97,10.57,9.04,0.99,31.05,0.11,18.10%,54.13%,1.07%,10.97%,12.57%,20.39,1.43
8,JOPA3,21.00,3.91,15.52,0.39,7.31,0.25,16.21%,5.65%,5.46%,2.51%,5.08%,5.44,1.78
9,JOPA4,33.80,2.67,24.98,0.63,7.31,0.40,16.21%,5.65%,5.46%,2.51%,5.08%,5.44,1.78
